<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/Welcome_To_Colaboratory1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation file
# path_to_data = '/content/fra.txt'

# # Read file
# translation_file = open(path_to_data,"r", encoding='utf-8') 
# raw_data = translation_file.read()
# translation_file.close()

# # Parse data
# raw_data = raw_data.split('\n')
# pairs = [sentence.split('\t') for sentence in  raw_data]
# pairs=pairs[0:60000]



In [ ]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [ ]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [ ]:

english_sentences = [clean_sentence(pair[0]) for pair in pairs]
french_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("french vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length french sentence: 14
Maximum length english sentence: 7
french vocabulary is of 14559 unique words
English vocabulary is of 6828 unique words


In [ ]:
print(french_sentences[50000])

n'est-ce pas agaçant


In [ ]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized,15, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized,15, padding = "post")




fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [ ]:
input_sequence = Input(shape=(15,))
embedding = Embedding(input_dim=french_vocab, output_dim=128,)(input_sequence)

In [ ]:
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(15)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)


In [ ]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 15)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1863552   
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 15, 64)           0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 15, 64)            33024     
                                                                 
 time_distributed_1 (TimeDis  (None, 15, 6828)         443820    
 tributed)                                                 

In [ ]:
BATCH_SIZE = 32
STEPS_PER_EPOCH = len(fra_pad_sentence) / BATCH_SIZE
SAVE_PERIOD = 10


from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model/weights1.{epoch:02d}.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH))

In [ ]:
print(int(SAVE_PERIOD * STEPS_PER_EPOCH))

20000


In [ ]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=32, epochs=70,callbacks=[checkpoint])


Epoch 1/70
1875/1875 [==============================] - 207s 109ms/step - loss: 1.5833 - accuracy: 0.7631
Epoch 2/70
1875/1875 [==============================] - 197s 105ms/step - loss: 1.5533 - accuracy: 0.7631
Epoch 3/70
1875/1875 [==============================] - 198s 106ms/step - loss: 1.5129 - accuracy: 0.7670
Epoch 4/70
1875/1875 [==============================] - 197s 105ms/step - loss: 1.3318 - accuracy: 0.7869
Epoch 5/70
1875/1875 [==============================] - 199s 106ms/step - loss: 1.1930 - accuracy: 0.8029
Epoch 6/70
1875/1875 [==============================] - 201s 107ms/step - loss: 1.0910 - accuracy: 0.8154
Epoch 7/70
1875/1875 [==============================] - 198s 105ms/step - loss: 1.0035 - accuracy: 0.8256
Epoch 8/70
1875/1875 [==============================] - 199s 106ms/step - loss: 0.9301 - accuracy: 0.8337
Epoch 9/70
1875/1875 [==============================] - 197s 105ms/step - loss: 0.8663 - accuracy: 0.8405
Epoch 10/70
1875/1875 [=======================

In [ ]:
enc_dec_model.save('my_model_v2.h5')

In [ ]:
import tensorflow as tf
new_enc_dec_model = tf.keras.models.load_model('my_model.h5')



In [ ]:
x=[french_sentences[50000]]
y=fra_text_tokenizer.texts_to_sequences(x)
z=pad_sequences(y, max_french_len, padding = "post")
print(fra_text_tokenizer.word_index['nestce'])
print(y)
print(z)

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 8000
# print("The english sentence is: {}".format(english_sentences[index]))
print("The french sentence is: {}".format(x))
print('The predicted sentence is :')
print(logits_to_sentence(new_model.predict(z)[0], eng_text_tokenizer))






In [ ]:
 # Lower case the sentence
lower_case_sent = "N'est-ce pas agaçan¡t!.".lower()
print(lower_case_sent)
# Strip punctuation
string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
print(string_punctuation)

clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
print(clean_sentence)
print(string.punctuation)

n'est-ce pas agaçan¡t!.
?¡¿.!
n'est-ce pas agaçant
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


exp

In [3]:
#exp

input_sequence1 = Input(shape=(12,))
embedding1 = Embedding(input_dim=7188, output_dim=128,)(input_sequence1)

model1 = Model(inputs=input_sequence1 , outputs=embedding1, name="mnist_model")
model1.compile('rmsprop', 'mse')


input_data = np.array([[1,2,4,5,18,0,0,0,4,10,12,13]])

output_array = model1.predict(input_data)
print(output_array[0][0])


[ 0.02034706  0.02248744 -0.03376602 -0.01687739 -0.03728447 -0.00209639
 -0.00100971 -0.02335948  0.00595356  0.02687982  0.03717594  0.02074369
 -0.04843444 -0.00176772 -0.00239968 -0.04342381  0.02203074  0.00309337
  0.00114974  0.04493412 -0.00245256 -0.017037   -0.04806034 -0.02295033
  0.04644542 -0.02189548  0.04477992  0.03860844  0.0291361   0.03252422
  0.02577506  0.0369927  -0.01474409  0.02011487  0.04395882  0.03625769
  0.02296979 -0.00022303 -0.04299204 -0.00083683 -0.00481214  0.03876538
 -0.04163405 -0.00957751 -0.00837046 -0.03523744  0.03373507 -0.02530587
 -0.00866864  0.02286681  0.00869872 -0.01245948  0.04525365  0.01941318
  0.04451017 -0.04003798 -0.02288283 -0.00895053 -0.03188037  0.02156677
  0.01939699 -0.04546351  0.03530386  0.02994759  0.04775039  0.03900445
  0.01326055 -0.00981707 -0.04240481 -0.021184   -0.04985572 -0.02521235
 -0.01867831  0.0159093   0.0180388  -0.02206373  0.04946562  0.04724665
  0.03032454 -0.02553148 -0.01261898  0.02932639 -0

In [ ]:
# for layer in model1.layers: print(layer.get_config(), layer.get_weights())
#print(model1.layers[1].get_weights()[0].shape)

(7188, 128)


In [4]:
#exp
lstm = LSTM(64, return_sequences=False)(embedding1)

lstmrepeatvector = RepeatVector(6)(lstm)
lstmdecoder = LSTM(64, return_sequences=True, dropout=0.2)(lstmrepeatvector)
lstmlogits = TimeDistributed(Dense(3733))(lstmdecoder)
modellstm = Model(inputs=input_sequence1 , outputs=lstmlogits, name="mnist_model")

In [5]:
#exp
modellstm.compile('rmsprop', 'mse')


In [6]:
modellstm.save('my_model_v21.h5')
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(modellstm, 'model')
!zip -r model.zip model

     |████████████████████████████████| 77 kB 3.1 MB/s 
     |████████████████████████████████| 462 kB 10.7 MB/s 
	zip warning: name not matched: content/model

zip error: Nothing to do! (try: zip -r model.zip . -i content/model)


In [ ]:
#EXP
output_array = modellstm.predict(input_data)
reshape_out=np.reshape(output_array[0][0],(3733,-1))
print(reshape_out.shape)


(3733, 1)


In [ ]:
# list_of_lists = [[1,2,3],[4,5,6],[7,8,9]]

# import csv

# with open("out.csv", "w") as f:
#     wr = csv.writer(f)
   
#     wr.writerows(exppairs)

exppairs=pairs[0:10]
file = open("sample.py", "w+")

# Saving the array in a text file
for pair in exppairs :
   file.write(str(pair))
file.close()


In [ ]:
# modelexp = tf.sequential();
# modelexp.add(tf.layers.dense({ units: 1, inputShape: [1] }));

In [46]:
import tensorflow as tf
import keras as keras
modelexp = Sequential()
# modelexp.add(Input(shape=(1,)))
modelexp.add(Dense(1,input_dim=1))

modelexp.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError( name="mean_squared_error"),metrics=['accuracy'])
xs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]);
xs=np.reshape(xs,(12,1))
ys = np.array([1, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]);
ys=np.reshape(ys,(12,1))
modelexp.fit(xs, ys,epochs=300, batch_size=1 )
x1=np.array([40])
x1=np.reshape(x1,(1,1))
print(x1.shape);
print(modelexp.predict(x1))

Epoch 1/300
12/12 [==============================] - 0s 2ms/step - loss: 35.2292 - accuracy: 0.0833 
Epoch 2/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1966 - accuracy: 0.0833
Epoch 3/300
12/12 [==============================] - 0s 2ms/step - loss: 0.3876 - accuracy: 0.0833
Epoch 4/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1449 - accuracy: 0.0833
Epoch 5/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1448 - accuracy: 0.0833
Epoch 6/300
12/12 [==============================] - 0s 1ms/step - loss: 0.1665 - accuracy: 0.0833
Epoch 7/300
12/12 [==============================] - 0s 1ms/step - loss: 0.1244 - accuracy: 0.0833
Epoch 8/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1186 - accuracy: 0.0833
Epoch 9/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1276 - accuracy: 0.0833
Epoch 10/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1088 - accuracy: 0.0833
Epoch 1

In [48]:
x1=np.array([40])
print(modelexp.predict(x1))

[[81.438965]]


In [ ]:
import tensorflow as tf
import keras as keras
modelexp = Sequential()
# modelexp.add(Input(shape=(1,)))
modelexp.add(Dense(1,input_dim=1))

modelexp.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError( name="mean_squared_error"),metrics=['accuracy'])
xs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]);
xs=np.reshape(xs,(12,1))
ys = np.array([1, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]);
ys=np.reshape(ys,(12,1))
modelexp.fit(xs, ys,epochs=300, batch_size=1 )
x1=np.array([40])
x1=np.reshape(x1,(1,1))
print(x1.shape);
print(modelexp.predict(x1))